# Project 1
## Dan & Josiah

In [ ]:
#Import necessary libraries
import numpy as np
import sklearn as sl
import pandas as pd 
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from google.colab import files
from math import sqrt
from sklearn import model_selection
from sklearn.feature_selection import RFE



In [121]:
# Load files 

#Select the files downloaded from the Kaggle Page
uploaded = files.upload()
train_df = pd.read_csv("train.csv")

uploaded = files.upload()
test_df = pd.read_csv("test.csv")


Saving train (1).csv to train (1) (2).csv


Saving test (1).csv to test (1) (2).csv


## START PRE-PROCESSING

In [122]:
'''
Let's take a look at the data sets.
'''
train_df.info()
test_df.info()

'''
The feature labels contain spaces.
We want to replce all the spaces with underscores.
'''
test_df.columns = [c.replace(' ', '_') for c in test_df.columns]
train_df.columns = [c.replace(' ', '_') for c in train_df.columns]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1758 entries, 0 to 1757
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               1758 non-null   int64  
 1   MS SubClass      1758 non-null   int64  
 2   MS Zoning        1758 non-null   object 
 3   Lot Frontage     1469 non-null   float64
 4   Lot Area         1758 non-null   int64  
 5   Street           1758 non-null   object 
 6   Alley            139 non-null    object 
 7   Lot Shape        1758 non-null   object 
 8   Land Contour     1758 non-null   object 
 9   Utilities        1758 non-null   object 
 10  Lot Config       1758 non-null   object 
 11  Land Slope       1758 non-null   object 
 12  Neighborhood     1758 non-null   object 
 13  Condition 1      1758 non-null   object 
 14  Condition 2      1758 non-null   object 
 15  Bldg Type        1758 non-null   object 
 16  House Style      1758 non-null   object 
 17  Overall Qual  

In [123]:
'''
We want to see where and how many NAs there are before we start filling in NAs.
This way, we will know beforehand if we want to fill NAs with zeros or some other calculated value.
'''
# Check for any missing values
null_train_df = pd.DataFrame(train_df.isnull().sum())
null_test_df = pd.DataFrame(test_df.isnull().sum())


'''
We see that there are a lot of missing values.
Can't fill these in with zeros - need to either use some averaging algorithm OR omit the feature(s) as a whole. 
'''
# Filter out all non-zeros 
null_train_df = null_train_df[null_train_df[0] != 0]
null_test_df = null_test_df[null_test_df[0] != 0]
null_test_df.head(10)



,0
Lot_Frontage,201
Alley,1113
Mas_Vnr_Type,13
Mas_Vnr_Area,13
Bsmt_Qual,36
Bsmt_Cond,36
Bsmt_Exposure,37
BsmtFin_Type_1,36
BsmtFin_Type_2,37
Electrical,1


In [124]:
'''
Manually separate the outcome feature from the rest of the training data set. 
'''
train_X = train_df.loc[:,train_df.columns!='SalePrice']
train_Y = pd.DataFrame(train_df['SalePrice'])
ids = pd.DataFrame(test_df["ID"]) # Save the ID column for submission.

#Data preprocessing
col_names_train = train_X.columns
# Repeat for test_df
col_names_test = test_df.columns



In [125]:
'''
First, drop all unneccesary training features (e.g., ID).
'''
drop_columns_train = list(null_train_df.index)
drop_columns_train.append("ID") 
train_X = train_X.drop(drop_columns_train, axis = 1)


In [126]:
'''
We want to extract dummy variables for all categorical features.
'''
# Automate the dummies
train_X = pd.get_dummies(train_X)
test_df = pd.get_dummies(test_df)

'''
We need the two data sets to be feature identical.
'''

common_cols = train_X.columns.intersection(test_df.columns) # Get all the common features 
train_X = train_X[common_cols]
test_df = test_df[common_cols]


In [127]:
'''
The test set may have other features that contain NAs - let's check. 
'''
null_test_df = pd.DataFrame(test_df.isnull().sum())
null_test_df = null_test_df[null_test_df[0] != 0]
null_test_df

'''
Garage_Cars and Garage Area each contain one entry with NAs.
Just drop these two records. 
'''
test_df = test_df.dropna()


In [128]:
'''
Standardize both the training and test sets. 
'''

#Import the scaler to beging standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
temp_norm = train_X.append(test_df)


col_names = temp_norm.columns

#Apply the scaler to the merged data set
min_max_scaler = preprocessing.MinMaxScaler()
temp_norm = min_max_scaler.fit_transform(temp_norm)
temp_norm = pd.DataFrame(temp_norm, columns=col_names)

temp_norm


,MS_SubClass,Lot_Area,Overall_Qual,Overall_Cond,Year_Built,Year_Remod/Add,1st_Flr_SF,2nd_Flr_SF,Low_Qual_Fin_SF,Gr_Liv_Area,Full_Bath,Half_Bath,Bedroom_AbvGr,Kitchen_AbvGr,TotRms_AbvGrd,Fireplaces,Garage_Cars,Garage_Area,Wood_Deck_SF,Open_Porch_SF,Enclosed_Porch,3Ssn_Porch,Screen_Porch,Pool_Area,Misc_Val,Mo_Sold,Yr_Sold,MS_Zoning_C (all),MS_Zoning_FV,MS_Zoning_I (all),MS_Zoning_RH,MS_Zoning_RL,MS_Zoning_RM,Street_Grvl,Street_Pave,Lot_Shape_IR1,Lot_Shape_IR2,Lot_Shape_IR3,Lot_Shape_Reg,Land_Contour_Bnk,...,Heating_Wall,Heating_QC_Ex,Heating_QC_Fa,Heating_QC_Gd,Heating_QC_TA,Central_Air_N,Central_Air_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,Kitchen_Qual_Ex,Kitchen_Qual_Fa,Kitchen_Qual_Gd,Kitchen_Qual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Typ,Paved_Drive_N,Paved_Drive_P,Paved_Drive_Y,Sale_Type_COD,Sale_Type_CWD,Sale_Type_Con,Sale_Type_ConLD,Sale_Type_ConLI,Sale_Type_ConLw,Sale_Type_New,Sale_Type_Oth,Sale_Type_WD,Sale_Condition_Abnorml,Sale_Condition_AdjLand,Sale_Condition_Alloca,Sale_Condition_Family,Sale_Condition_Normal,Sale_Condition_Partial
0,0.235294,0.045077,0.666667,0.50,0.884058,0.733333,0.186725,0.437772,0.0,0.337792,0.50,0.5,0.375,0.333333,0.230769,0.50,0.4,0.352823,0.120084,0.177898,0.000000,0.0,0.000000,0.0,0.000000,0.636364,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.235294,0.050719,0.888889,0.50,0.963768,0.916667,0.226843,0.444068,0.0,0.376225,0.50,0.5,0.375,0.333333,0.538462,0.25,0.6,0.674059,0.134831,0.084906,0.000000,0.0,0.000000,0.0,0.000000,0.636364,0.50,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.588235,0.010778,0.666667,0.50,0.971014,0.950000,0.213821,0.000000,0.0,0.191786,0.50,0.0,0.250,0.333333,0.230769,0.25,0.4,0.313172,0.000000,0.324798,0.000000,0.0,0.000000,0.0,0.000000,0.636364,0.25,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.058824,0.036449,0.333333,0.75,0.347826,0.866667,0.056921,0.000000,0.0,0.051055,0.25,0.0,0.250,0.333333,0.230769,0.00,0.0,0.000000,0.000000,0.000000,0.142292,0.0,0.000000,0.0,0.000000,0.545455,0.25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.027577,0.333333,0.50,0.565217,0.000000,0.111321,0.000000,0.0,0.099849,0.25,0.0,0.250,0.333333,0.230769,0.00,0.2,0.354839,0.000000,0.000000,0.000000,0.0,0.199653,0.0,0.000000,0.636364,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,0.411765,0.026726,0.444444,0.50,0.775362,0.483333,0.252258,0.000000,0.0,0.226262,0.50,0.0,0.500,0.666667,0.461538,0.00,0.4,0.268817,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.454545,0.75,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2925,0.147

In [129]:
'''
Split the set back into the train and test sets.

'''
train_X = temp_norm.iloc[:1758]
test_df = temp_norm.iloc[1758:]

In [130]:
# Make Linear Regression Model 
model = LinearRegression(fit_intercept=True)
model.fit(train_X, train_Y)
pred = model.predict(test_df)
print(pred)


[[ 80067.]
 [161169.]
 [184493.]
 ...
 [205734.]
 [280357.]
 [288391.]]


In [131]:
'''
We have the predictions.
Need to make the final data set output. 
'''
final_dataset = pd.DataFrame(pred, columns = ['SalePrice'])
final_dataset = pd.concat([ids, final_dataset], axis = 1)
final_dataset = final_dataset[["ID", "SalePrice"]]
final_dataset.head(10)

,ID,SalePrice
0,1758,80067.0
1,1759,161169.0
2,1760,184493.0
3,1761,120833.0
4,1762,284773.0
5,1763,491199.0
6,1764,127861.0
7,1765,78387.0
8,1766,152079.0
9,1767,127153.0


In [132]:
# Make a new df with the actual and predicted variables 

compare = pd.concat([final_dataset["SalePrice"], train_Y],axis = 1)
compare = compare.rename(columns={"SalePrice":"Predicted", "SalePrice":"Ac"})


In [133]:
final_dataset.tail(10)

,ID,SalePrice
1162,2920,1.054810e+05
1163,2921,1.075170e+05
1164,2922,8.883700e+04
1165,2923,-5.444333e+15
1166,2924,1.208570e+05
1167,2925,1.285410e+05
1168,2926,2.057340e+05
1169,2927,2.803570e+05
1170,2928,2.883910e+05
1171,2929,NaN


In [113]:
#This will export our previous table as a csv for submission.
final_dataset.to_csv('Submission.csv')
from google.colab import files
files.download("Submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>